# Description

# Setup

In [1]:
import pandas as pd
import requests
import plotly.express as px
from ons_api import get_list_of_datasets, get_dataset_by_name, get_edition, get_timeseries
import io
import datetime

# Data

## BR

In [44]:
url = "https://api.bcb.gov.br/dados/serie/bcdata.sgs.433/dados?formato=csv&dataInicial=01/01/1995&dataFinal=31/12/2025"
df_br = pd.read_csv(url, sep=";", decimal=",", date_format="%d/%m/%Y", parse_dates=[0])
df_br['12m_ipca'] = (df_br['valor']/100 + 1).rolling(12).apply(lambda x: x.prod()) - 1
df_br.head()

,data,valor,12m_ipca
0,1995-01-01,1.70,NaN
1,1995-02-01,1.02,NaN
2,1995-03-01,1.55,NaN
3,1995-04-01,2.43,NaN
4,1995-05-01,2.67,NaN


In [45]:
fig = px.line(df_br,
              x='data',
              y='12m_ipca',
              template='none',
              width=None)
fig

## UK

In [30]:
dss = get_list_of_datasets()
dataset_name = "Consumer Prices Index including owner occupiers' housing costs (CPIH)"
dataset = get_dataset_by_name(dss, dataset_name)
edition_url = get_edition(dataset)
r = requests.get(edition_url)
results = r.json()
latest_edition_url = results.get("downloads").get("csv").get("href")
latest_edition_url
r = requests.get(latest_edition_url).content
df_uk = pd.read_csv(io.BytesIO(r), date_format="%b-%y", parse_dates=[1,2])
df_uk.head()

,v4_0,mmm-yy,Time,uk-only,Geography,cpih1dim1aggid,Aggregate
0,147.1,2025-05-01,2025-05-01,K02000001,United Kingdom,CP0122,"01.2.2 Mineral waters, soft drinks and juices"
1,138.9,2025-05-01,2025-05-01,K02000001,United Kingdom,CP051,"05.1 Furniture, furnishings and carpets"
2,138.7,2025-05-01,2025-05-01,K02000001,United Kingdom,CP0721,07.2.1 Spare parts and accessories
3,162.9,2025-05-01,2025-05-01,K02000001,United Kingdom,CP0733,07.3.3 Passenger transport by air
4,175.4,2025-05-01,2025-05-01,K02000001,United Kingdom,CP081,08.1 Postal services


In [27]:
df_uk['Aggregate'].sort_values().drop_duplicates()

45306    01 Food and non-alcoholic beverages
3307                               01.1 Food
34823               01.1.1 Bread and cereals
38210                            01.1.2 Meat
37455                            01.1.3 Fish
                        ...                 
2694                 12.5.3 Health insurance
16563             12.5.4 Transport insurance
12955          12.6 Financial services (nec)
36575              12.7 Other services (nec)
2774                           Overall Index
Name: Aggregate, Length: 126, dtype: object

In [31]:
df_uk[df_uk['Aggregate']=="Overall Index"]

,v4_0,mmm-yy,Time,uk-only,Geography,cpih1dim1aggid,Aggregate
101,138.0,2025-05-01,2025-05-01,K02000001,United Kingdom,CP00,Overall Index
166,137.7,2025-04-01,2025-04-01,K02000001,United Kingdom,CP00,Overall Index
284,136.1,2025-03-01,2025-03-01,K02000001,United Kingdom,CP00,Overall Index
466,135.6,2025-02-01,2025-02-01,K02000001,United Kingdom,CP00,Overall Index
506,135.1,2025-01-01,2025-01-01,K02000001,United Kingdom,CP00,Overall Index
...,...,...,...,...,...,...,...
54252,48.0,1988-05-01,1988-05-01,K02000001,United Kingdom,CP00,Overall Index
54337,47.8,1988-04-01,1988-04-01,K02000001,United Kingdom,CP00,Overall Index
54442,47.2,1988-03-01,1988-03-01,K02000001,United Kingdom,CP00,Overall Index
54578,47.0,1988-02-01,1988-02-01,K02000001,United Kingdom,CP00,Overall Index


In [34]:
data_chart = df_uk[df_uk['Aggregate']=="Overall Index"]

fig = px.line(
    data_chart,
    x='Time',
    y='v4_0',
    width=700,
    template='none'
)
fig 

## World Bank

In [38]:
from zipfile import ZipFile

In [49]:
cpi_source = 'https://api.worldbank.org/v2/en/indicator/FP.CPI.TOTL.ZG?downloadformat=csv'
r = requests.get(cpi_source).content
zip_file = ZipFile(io.BytesIO(r))
zip_file.infolist()
pd.read_csv(zip_file.open("API_FP.CPI.TOTL.ZG_DS2_en_csv_v2_80938.csv"), 
            encoding='ISO-8859-1', 
            skiprows=4)

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2016,2017,2018,2019,2020,2021,2022,2023,2024,Unnamed: 69
0,Aruba,ABW,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.931196,-1.028282,3.626041,4.257462,NaN,NaN,NaN,NaN,NaN,NaN
1,Africa Eastern and Southern,AFE,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,6.596505,6.399343,4.720805,4.644967,5.405162,7.240978,10.773751,7.126975,NaN,NaN
2,Afghanistan,AFG,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,4.383892,4.975952,0.626149,2.302373,5.601888,5.133203,13.712102,-4.644709,-6.601186,NaN
3,Africa Western and Central,AFW,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,1.487416,1.725486,1.784050,1.983092,2.490378,3.745700,7.949251,5.221168,NaN,NaN
4,Angola,AGO,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,30.694415,29.844480,19.628938,17.080954,22.271539,25.754295,21.355290,13.644102,28.240495,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,Kosovo,XKX,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,0.273169,1.488234,1.053798,2.675992,0.198228,3.353691,11.580510,4.944324,1.619450,NaN
262,"Yemen, Rep.",YEM,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
263,South Africa,ZAF,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,1.288877,2.102351,1.246275,1.337971,2.534989,4.069019,...,6.602908,5.186187,4.509873,4.102851,3.232388,4.618745,7.039873,6.075244,4.361152,NaN
264,Zambia,ZMB,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,17.869730,6.577312,7.494572,9.150316,15.733060,22.020768,10.993204,10.884532,NaN,NaN


## IMF

In [18]:
# import libraries
import sdmx

# retrieve data
IMF_DATA = sdmx.Client('IMF_DATA')
data_msg = IMF_DATA.data('CPI', key='USA+CAN+BRA+MEX+COL+CHI+GBR.CPI.CP01.IX.M', params={'startPeriod': 2018})

cpi_df = sdmx.to_pandas(data_msg)
cpi_df = pd.DataFrame(cpi_df).reset_index()
cpi_df['COUNTRY'].drop_duplicates()

xml.Reader got no structure=… argument for StructureSpecificData


0      BRA
88     CAN
176    COL
265    GBR
354    MEX
440    USA
Name: COUNTRY, dtype: object

In [20]:
cpi_df[cpi_df['COUNTRY']=="BRA"]

,COUNTRY,INDEX_TYPE,COICOP_1999,TYPE_OF_TRANSFORMATION,FREQUENCY,OVERLAP,SCALE,ACCESS_SHARING_LEVEL,SECURITY_CLASSIFICATION,COMMON_REFERENCE_PERIOD,TIME_PERIOD,value
0,BRA,CPI,CP01,IX,M,OL,0,PUBLIC_OPEN,PUB,2018-M01,NaN,NaN
1,BRA,CPI,CP01,IX,M,OL,0,PUBLIC_OPEN,PUB,2018-M02,NaN,NaN
2,BRA,CPI,CP01,IX,M,OL,0,PUBLIC_OPEN,PUB,2018-M03,NaN,NaN
3,BRA,CPI,CP01,IX,M,OL,0,PUBLIC_OPEN,PUB,2018-M04,NaN,NaN
4,BRA,CPI,CP01,IX,M,OL,0,PUBLIC_OPEN,PUB,2018-M05,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
83,BRA,CPI,CP01,IX,M,OL,0,PUBLIC_OPEN,PUB,2024-M12,NaN,NaN
84,BRA,CPI,CP01,IX,M,OL,0,PUBLIC_OPEN,PUB,2025-M01,NaN,NaN
85,BRA,CPI,CP01,IX,M,OL,0,PUBLIC_OPEN,PUB,2025-M02,NaN,NaN
86,BRA,CPI,CP01,IX,M,OL,0,PUBLIC_OPEN,PUB,2025-M03,NaN,NaN


In [33]:
url = 'http://dataservices.imf.org/REST/SDMX_JSON.svc/'
param = [('dataset', 'IFS'),
         ('freq', 'M'),
         ('country', 'BR+MX+CL+CO+GB+US+JP+CH+DE'),
         ('series', 'PCPI_IX'),
         ('start', '?startPeriod=1995')]
series = '.'.join([i[1] for i in param[1:4]])

key = f'CompactData/{param[0][1]}/{series}{param[-1][1]}'
r = requests.get(f'{url}{key}')
r_json = r.json()
data = r_json['CompactData']['DataSet']['Series']
# Create pandas dataframe, column = country, row = obs
df = pd.DataFrame({s['@REF_AREA']: # Each country/area
                   {i['@TIME_PERIOD']: float(i['@OBS_VALUE']) 
                    for i in s['Obs']} for s in data})

df.index = pd.to_datetime(df.index)
df = df.reset_index(names="date")
df = df.melt(id_vars='date', var_name="country", value_name="cpi").sort_values(['country', 'date'])
df.head()

,date,country,cpi
1460,1995-01-01,BR,33.191158
1461,1995-02-01,BR,33.529575
1462,1995-03-01,BR,34.049401
1463,1995-04-01,BR,34.876820
1464,1995-05-01,BR,35.807947


In [34]:
df['country'].drop_duplicates()

1460    BR
2190    CH
730     CL
0       CO
2555    DE
1095    GB
365     JP
1825    MX
2920    US
Name: country, dtype: object

In [ ]:
df['monthly_cpi'] = df.groupby("country")['cpi'].pct_change(fill_method=None)
df['cum_cpi'] = (df.groupby("country")['monthly_cpi']).transform(lambda x: (1+x).cumprod()) - 1
df['cum_cpi_12'] = df.groupby("country")['monthly_cpi'].transform(lambda s:s.rolling(12).apply(lambda x: (1+x).prod()))-1
df

,date,country,cpi,monthly_cpi,cum_cpi,cum_cpi_12
1460,1995-01-01,BR,33.191158,NaN,NaN,NaN
1461,1995-02-01,BR,33.529575,0.010196,0.010196,NaN
1462,1995-03-01,BR,34.049401,0.015504,0.025858,NaN
1463,1995-04-01,BR,34.876820,0.024301,0.050786,NaN
1464,1995-05-01,BR,35.807947,0.026698,0.078840,NaN
...,...,...,...,...,...,...
2915,2025-01-01,US,145.683553,0.006546,1.113580,0.030005
2916,2025-02-01,US,146.330636,0.004442,1.122967,0.028215
2917,2025-03-01,US,146.659451,0.002247,1.127738,0.023907
2918,2025-04-01,US,147.116216,0.003114,1.134365,0.023113


# analysis

In [21]:
data_chart = df.copy()
fig = px.line(
    data_chart,
    x='date',
    y='cum_cpi',
    color='country',
    width=700,
    template='none'
)
fig 

In [22]:
data_chart = df.copy()
data_chart = data_chart[data_chart['date'] >= datetime.datetime(2000,1,1)]
fig = px.line(
    data_chart,
    x='date',
    y='cum_cpi_12',
    color='country',
    width=700,
    template='none'
)
fig 

In [26]:
data_chart = df.copy()
data_chart = data_chart[data_chart['date'] >= datetime.datetime(2000,1,1)]
data_chart['cum_cpi_XX'] = data_chart\
                            .groupby("country")['monthly_cpi']\
                            .transform(lambda s:s.rolling(48)\
                                       .apply(lambda x: (1+x).prod()))-1
fig = px.line(
    data_chart,
    x='date',
    y='cum_cpi_XX',
    color='country',
    width=700,
    template='none'
)
fig 